## import and download dataset and library

In [1]:
!git clone https://github.com/nkgrush/SSYP2022-ws8
%cd SSYP2022-ws8
!ls data

fatal: destination path 'SSYP2022-ws8' already exists and is not an empty directory.
/content/SSYP2022-ws8
2018-06-06-pdb-intersect-pisces.csv.zip  2018-06-06-ss.cleaned.csv.zip


In [2]:
!unzip data/2018-06-06-pdb-intersect-pisces.csv.zip
!unzip data/2018-06-06-ss.cleaned.csv.zip

Archive:  data/2018-06-06-pdb-intersect-pisces.csv.zip
replace 2018-06-06-pdb-intersect-pisces.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2018-06-06-pdb-intersect-pisces.csv  
Archive:  data/2018-06-06-ss.cleaned.csv.zip
replace 2018-06-06-ss.cleaned.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2018-06-06-ss.cleaned.csv  


In [3]:
!pip install torchtext torchdata

import numpy as np
from functools import partial

from tqdm.notebook import tqdm, trange

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import torch
import torchtext
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import WikiText2, WikiText103

import random
from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from tqdm.notebook import tqdm, trange

import pandas as pd

import matplotlib.pyplot as plt



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## get data

In [4]:
df = pd.read_csv('./2018-06-06-pdb-intersect-pisces.csv')

In [5]:
df

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa,Exptl.,resolution,R-factor,FreeRvalue
0,1FV1,F,NPVVHFFKNIVTPRTPPPSQ,CCCCCBCCCCCCCCCCCCCC,CCCCCECCCCCCCCCCCCCC,20,False,XRAY,1.90,0.23,0.27
1,1LM8,H,DLDLEMLAPYIPMDDDFQLR,CCCCCCCCCBCCSCCCEECC,CCCCCCCCCECCCCCCEECC,20,False,XRAY,1.85,0.20,0.24
2,1O06,A,EEDPDLKAAIQESLREAEEA,CCCHHHHHHHHHHHHHHHTC,CCCHHHHHHHHHHHHHHHCC,20,False,XRAY,1.45,0.19,0.22
3,1QOW,D,CTFTLPGGGGVCTLTSECI*,CCTTSCTTCSSTTSSTTCCC,CCCCCCCCCCCCCCCCCCCC,20,True,XRAY,1.06,0.14,1.00
4,1RDQ,I,TTYADFIASGRTGRRNAIHD,CHHHHHHTSSCSSCCCCEEC,CHHHHHHCCCCCCCCCCEEC,20,False,XRAY,1.26,0.13,0.16
...,...,...,...,...,...,...,...,...,...,...,...
9073,4LGY,A,GDGLVPRGSHMMEILRGSPALSAFRINKLLARFQAANLQVHNIYAE...,CCEEEEETTEEEEEEEEEECCCHHHHHHHHHHHHHTTCCCCEEEEE...,CCEEEEECCEEEEEEEEEECCCHHHHHHHHHHHHHCCCCCCEEEEE...,1305,False,XRAY,1.48,0.14,0.17
9074,5XH6,A,GSHMTQFEGFTNLYQVSKTLRFELIPQGKTLKHIQEQGFIEEDKAR...,CCCCCCGGGCBSSSCEEEEEEEEEEECTTHHHHHHHHTHHHHHHHH...,CCCCCCHHHCECCCCEEEEEEEEEEECCCHHHHHHHHCHHHHHHHH...,1310,False,XRAY,2.00,0.18,0.21
9075,5B2R,B,GSGHMDKKYSIGLAIGTNSVGWAVITDEYKVPSKKFKVLGNTDRHS...,CCCCCCCCCEEEEEECSSEEEEEEECTTSCCCEEEEEEEESSSCCE...,CCCCCCCCCEEEEEECCCEEEEEEECCCCCCCEEEEEEEECCCCCE...,1372,False,XRAY,2.00,0.20,0.23
9076,5WLH,A,SNAMKISKVREENRGAKLTVNAKTAVVSENRSQEGILYNDPSRYGK...,CCCCCCCCCCCCCCCCCTTSSEEEEEEESSSCEEEEEEESSCCTTT...,CCCCCCCCCCCCCCCCCCCCCEEEEEEECCCCEEEEEEECCCCCCC...,1440,False,XRAY,1.80,0.19,0.22


In [6]:
seq = df['seq'].to_numpy()
sst3 = df['sst3'].to_numpy()

In [7]:
seq.shape, sst3.shape

((9078,), (9078,))

## tokenizer

In [8]:
def tokenizer(seq):
    return list(seq)

## create custom dataset


In [9]:
class SecondaryStructureDataset(Dataset):
    def __init__(self, sequence, secondary_struct):
        super().__init__()
        idx = self.filter_sequences(sequence)
        self.sequence = sequence[idx]
        self.secondary_struct = secondary_struct[idx]
        self.sequence = self.process_sequences(self.sequence)
        self.secondary_struct = self.process_sequences(self.secondary_struct)
        #secondary_struct = self.process_sequences(secondary_struct)
        self._len = len(self.sequence)
    def __len__(self):
        return self._len
    
    def __getitem__(self, index):
        return self.sequence[index], self.secondary_struct[index]

    def process_sequences(self, seqs):
        seqs = np.array([s[:201] for s in seqs])
        return seqs

    def filter_sequences(self, seqs, cutoff=200):
        lens = np.array([ len(s) for s in seqs ])
        #print(lens)
        return lens >= cutoff

In [10]:
dataset = SecondaryStructureDataset(seq, sst3)
len(dataset)

4772

In [11]:
np.vectorize(len)( np.array(['1', '2ffffa']) )

array([1, 6])

In [12]:
dataset[786]

('GSNADMEMTLERAVSMLEADHMLPSRISAAATFIQHECFQKSEARKRVNQLRGILKLLQLLKVQNEDVQRAVCGALRNLVFEDNDNKLEVAELNGVPRLLQVLKQTRDLETKKQITGLLWNLSSNDKLKNLMITEALLTLTENIIIPFSGWPEGDYPKANGLLDFDIFYNVTGCLRNMSSAGADGRKAMRRCDGLIDSLVH',
 'CCCCCCCCCHHHHHHCCCCCCCCHHHHHHHHHHHHHHHHHCHHHHHHHHHCCHHHHHHHHHHCCCHHHHHHHHHHHHHHHCCCHHHHHHHHHCCHHHHHHHHHHHCCCHHHHHHHHHHHHHHHCCHHHHHHHHHHHHHHHHHHCHHHHHCCCHHHCCCCCCCCCHHHHHHHHHHHHHHCCCCHHHHHHHHCCCCHHHHHHH')

## create batch_sampler


In [13]:
class BatchSampler():
    def __init__(self, dataset, batch_size, tokenizer):
        lens = []
        self.idx = []
        self.batch_size = batch_size
        for x, y in dataset:
            x = tokenizer(x)
            lens.append(len(x))
        idx = np.arange(0, len(lens))

        for i in range(0, len(idx), batch_size * 100):
            sorted_idx = sorted(idx[i:i + batch_size * 100], key=lambda x: lens[x])
            self.idx.extend(sorted_idx)
        
        self.__len = len(idx) // batch_size # длина в батчах

    def __len__(self):
        return self.__len

    def __iter__(self):
        batch_size = self.batch_size
        for batch_idx in range(0, self.__len):
            indices = self.idx[batch_idx * batch_size:(batch_idx + 1) * batch_size]
            assert len(indices) == self.batch_size
            yield indices

## build vocab

In [14]:
vocab = build_vocab_from_iterator(
        map(lambda seq_label: tokenizer(seq_label[0]), dataset),
        specials=["<unk>", "<pad>"],
    )
vocab.set_default_index(vocab["<unk>"])
PAD_IDX = vocab['<pad>']

In [15]:
vocab.get_stoi()

{'*': 22,
 '<pad>': 1,
 '<unk>': 0,
 'A': 3,
 'C': 21,
 'D': 8,
 'E': 7,
 'F': 15,
 'G': 4,
 'H': 18,
 'I': 9,
 'K': 11,
 'L': 2,
 'M': 19,
 'N': 14,
 'P': 13,
 'Q': 16,
 'R': 12,
 'S': 6,
 'T': 10,
 'V': 5,
 'W': 20,
 'Y': 17}

## pipelines

In [16]:
def text_pipeline(seq):
    seq = tokenizer(seq)
    idx = vocab(seq)
    return idx

label_vocab = {
    'C': 0,
    'E': 1,
    'H': 2
}

text_pipeline = lambda x: vocab(tokenizer(x))
def label_pipeline(label_seq):
    label_pipeline_list = []
    for char in tokenizer(label_seq):
        label_pipeline_list.append(label_vocab[char])
    return label_pipeline_list

In [17]:
(lambda x, b: x**2+b)(5, 2)

27

## collate_fn

In [18]:
# попробуй сегодня

MAX_LEN = None
def collate_fn(batch, text_pipeline=text_pipeline):
    batch_input, batch_output = [], []
    min_len = 9999999999999
    for seq, label in batch:
        text_tokens_ids = text_pipeline(seq)
        label_ids = label_pipeline(label)

        if MAX_LEN is not None:
            text_tokens_ids = text_tokens_ids[:MAX_LEN]
        
        curr_len = len(text_tokens_ids)
        if curr_len < min_len:
            min_len = curr_len

        # узнай минимальную длину из text_tokens_ids
        # обрежь все inputs до этой минимальной длины
        # обрежь все outputs до этой минимальной длины

        batch_input.append(torch.LongTensor(text_tokens_ids))
        batch_output.append(torch.LongTensor(label_ids))
    # перед тем как создавать тензоры
    
    #print(min_len)
    batch_input = [value[:min_len] for value in batch_input ]
    batch_output = [value[:min_len] for value in batch_output ]

    original_lens = [len(s) for s in batch_input]
    batch_input = pad_sequence(batch_input, padding_value=PAD_IDX)

    batch_input = torch.tensor(batch_input, dtype=torch.long)
    batch_output = torch.stack(batch_output)

    return batch_input, batch_output, torch.tensor(original_lens)

## dataloader

In [19]:
batch_size=16
dataloader = DataLoader(
        dataset,
        batch_sampler=BatchSampler(dataset, batch_size, tokenizer),
        collate_fn=collate_fn,
)

In [20]:
for batch in dataloader:
    #print(batch)
    pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


## define model

In [21]:
# use nn.LSTM 
class Model(nn.Module):
    def __init__(self, vocab_size, classes, input_dim, hid_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, input_dim)
        self.LSTM = nn.LSTM(input_dim, hid_dim, num_layers=2, bidirectional=True, dropout=0.1)
        self.fc = nn.Linear(hid_dim*2, classes)
        
    def forward(self, x, original_lens):
        #print(x.shape)
        x = self.emb(x)
        #print(x.shape)
        out, (hid, cell) = self.LSTM(x)
        x = self.fc(out)
        return x

batch = next(iter(dataloader))
seq, target, original_lens = batch
batch 

model = Model(len(vocab), 3, 50, 50)
out = model(seq, original_lens)
out.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


torch.Size([200, 16, 3])

In [22]:
a = torch.rand(5, 4, 2)
b = torch.rand(5, 4, 2)

torch.cat((a,b), dim=-1)

tensor([[[0.5258, 0.1057, 0.5175, 0.5910],
         [0.1641, 0.3748, 0.6926, 0.2872],
         [0.2923, 0.5321, 0.9752, 0.9195],
         [0.3038, 0.1801, 0.5802, 0.6404]],

        [[0.8907, 0.6773, 0.2828, 0.3755],
         [0.8976, 0.8417, 0.8987, 0.6352],
         [0.4043, 0.4997, 0.7138, 0.2559],
         [0.4596, 0.6199, 0.3602, 0.5679]],

        [[0.4191, 0.7692, 0.5334, 0.9419],
         [0.4548, 0.4991, 0.9894, 0.1098],
         [0.9226, 0.0865, 0.5078, 0.9000],
         [0.6275, 0.4826, 0.0644, 0.4541]],

        [[0.6086, 0.6656, 0.9723, 0.4641],
         [0.5502, 0.5873, 0.0945, 0.6326],
         [0.0852, 0.0522, 0.6871, 0.1260],
         [0.5520, 0.1465, 0.5792, 0.7860]],

        [[0.4326, 0.1522, 0.4908, 0.7805],
         [0.6172, 0.4453, 0.7756, 0.4621],
         [0.6902, 0.0756, 0.3683, 0.3054],
         [0.3632, 0.9404, 0.2350, 0.4177]]])

## define metrics

## define train loop and optimizer

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 3
def train(model, optimizer, criterion):
    model.train()
    model.to(device)
    acc_hist = []
    loss_hist = []
    
    top_bar = trange(1, 100)
    for e in top_bar:
        epoch_loss = 0
        epoch_acc = 0
        i = 0
        epoch_bar = tqdm(dataloader)
        for x_b, y_b, original_lens in epoch_bar:
            x_b = x_b.to(device)
            y_b = y_b.to(device)

            optimizer.zero_grad()
            y_pred = model(x_b, original_lens)

            random_value = [random.randint(100, 190)]
            y_pred = y_pred[random_value, :, :].reshape(-1, NUM_CLASSES)
            y_b = y_b[:, random_value].reshape(-1)
            loss = criterion(y_pred, y_b)
            acc = sum(y_pred.argmax(-1) == y_b) / len(y_b)

            loss.backward()
            optimizer.step()

            loss = loss.cpu().item()
            acc = acc.cpu().item()

            loss_hist.append(loss)
            acc_hist.append(acc)
            epoch_loss += loss
            epoch_acc += acc
            i += 1

            epoch_bar.set_description(f'loss: {loss:.4f}, acc: {acc:.4f}')

        epoch_loss /= i
        epoch_acc /= i
        top_bar.set_description(f'loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}')
    return loss_hist, acc_hist

model = Model(len(vocab), 3, 50, 50)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss() 
loss_hist, acc_hist = train(model, optimizer, criterion)

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

...

  0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

In [ ]:
model(seq.to(device), original_lens)

In [ ]:
plt.plot(loss_hist)

In [ ]:
plt.plot(acc_hist)

## train model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss() 
train(model, optimizer, criterion)

## eval and check metrics


## Save and then try something else